In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline

In [2]:
# veri kaynağı: http://veribilimi.co/data/iris.csv
df = pd.read_csv("http://veribilimi.co/data/iris.csv")

# Veri keşfi

In [4]:
df.groupby(['Species']).agg(['mean']).head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
,mean,mean,mean,mean
Species,,,,
Iris-setosa,5.006,3.418,1.464,0.244
Iris-versicolor,5.936,2.770,4.260,1.326
Iris-virginica,6.588,2.974,5.552,2.026


In [3]:
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
X = df.iloc[:,:-1].values
X.shape

(150, 4)

In [5]:
y = df.iloc[:,-1].values
y.shape

(150,)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=142)

In [8]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(112, 4)
(38, 4)
(112,)
(38,)


In [9]:
from sklearn.neighbors import KNeighborsClassifier

In [10]:
classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
# minkowski: mesafe metriği
# n_neighbors: kaç komşuya bakılıp kararverilecek, tek sayı olmalı
# p: eğer minkowski seçersek hangi yöntemi olsun

In [11]:
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [12]:
y_pred = classifier.predict(X_test)

## Tahmin sonuçlarını numpy.ndarray'den dataframe yapma (İncelemek daha Kolay)

In [13]:
predictions = pd.DataFrame(data=y_pred,    # values
                index=range(len(y_pred)),    # 1st column as index
                   columns=['y_pred'])  # 1st row as the column names

# Sadece y_pred'den oluşan df'e test(gerçek) y_test'i sütun olarak ekleme
predictions['y_test'] = y_test
predictions.head()

,y_pred,y_test
0,Iris-setosa,Iris-setosa
1,Iris-versicolor,Iris-versicolor
2,Iris-versicolor,Iris-versicolor
3,Iris-virginica,Iris-virginica
4,Iris-versicolor,Iris-versicolor


# Hata Matrisi (Confusion Matrix)

In [14]:
from sklearn.metrics import confusion_matrix

In [16]:
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(cm)

[[ 7  0  0]
 [ 0 14  3]
 [ 0  1 13]]


In [17]:
from sklearn.metrics import accuracy_score

In [18]:
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: % {:10.2f}".format(accuracy*100)) 
# :10.2f ifadesi ondalık kısımda iki basamak göstermek için

Accuracy: %      89.47


# Elle Accuracy hesaplama

In [19]:
predictions[predictions['y_pred'] != predictions['y_test']]

,y_pred,y_test
14,Iris-virginica,Iris-versicolor
18,Iris-versicolor,Iris-virginica
19,Iris-virginica,Iris-versicolor
33,Iris-virginica,Iris-versicolor


In [20]:
predictions[predictions['y_pred'] != predictions['y_test']].count()

y_pred    4
y_test    4
dtype: int64

In [21]:
len(y_test)

38

In [22]:
accuracy = (len(y_test) - 4) / len(y_test)
accuracy

0.8947368421052632

# En iyi k değeri

In [33]:
accuracy_list = []
k_list = []
for k in range(2,10):
    classifier = KNeighborsClassifier(n_neighbors=k, metric='minkowski', p=2)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_list.append(accuracy)
    k_list.append(k)
    print("{} için accuracy: {:10.2f}".format(k,accuracy))
    
    
    
    

2 için accuracy:       0.89
3 için accuracy:       0.92
4 için accuracy:       0.92
5 için accuracy:       0.89
6 için accuracy:       0.95
7 için accuracy:       0.95
8 için accuracy:       0.95
9 için accuracy:       0.95


In [34]:
acc_max_index = accuracy_list.index(max(accuracy_list))
acc_max_index

4

In [35]:
k_list[acc_max_index]

6

# En iyi k değeri için modeli eğitelim

In [36]:
# find index of max value inaccuracy_list
acc_max_index = accuracy_list.index(max(accuracy_list))
# assing this index to k_list
k = k_list[acc_max_index]
classifier = KNeighborsClassifier(n_neighbors=k, metric='minkowski', p=2)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=6, p=2,
           weights='uniform')

# En iyi modelle tahmin yapma

In [37]:
# test setinden tahminler yapalım
y_pred = classifier.predict(X_test)

In [38]:
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: % {:10.2f}".format(accuracy*100)) 
# :10.2f ifadesi ondalık kısımda iki basamak göstermek için

Accuracy: %      94.74


# Elle tahmin seti üretme

In [29]:
# test seti dışında elle tahmin seti oluşturalım
X_manual_test = np.array([4.0,4.0,4.0,4.0]).reshape(1,4)

# array shape mutlaka uygun olmalı
print(X_manual_test.shape)
print(X_test.shape)

(1, 4)
(38, 4)


In [30]:
classifier.predict(X_manual_test)

array(['Iris-versicolor'], dtype=object)

# Hata Matrisi

In [31]:
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(cm)

[[ 7  0  0]
 [ 0 17  0]
 [ 0  2 12]]


# y_pred ve y_test'i dataframe formatında inceleme

In [32]:
predictions = pd.DataFrame(data=y_pred, 
                index=range(len(y_pred)),    # y_pred uzunluğunca int index ver
                   columns=['y_pred'])  # sütun ismi

# Sadece y_pred'den oluşan df'e test(gerçek) y_test'i sütun olarak ekleme
predictions['y_test'] = y_test
predictions.head(15)

,y_pred,y_test
0,Iris-setosa,Iris-setosa
1,Iris-versicolor,Iris-versicolor
2,Iris-versicolor,Iris-versicolor
3,Iris-virginica,Iris-virginica
4,Iris-versicolor,Iris-versicolor
5,Iris-versicolor,Iris-versicolor
6,Iris-setosa,Iris-setosa
7,Iris-setosa,Iris-setosa
8,Iris-virginica,Iris-virginica
9,Iris-versicolor,Iris-versicolor


## Acaba ürettiğimiz dataframe de bir eksiklik var mı?

In [82]:
print(len(y_test))
print(len(predictions['y_pred']))

38
38
